In [1]:
# -*- coding: utf-8 -*-
# 作者: xcl
# 时间: 2019/8/1 21:51
import pandas as pd
import keras
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, core, add
from keras.models import Model
import os

Using TensorFlow backend.


In [127]:
# 读取
input_path = 'D:\\毕业论文程序\\建模数据_合并\\全2018.xlsx'
data_all = pd.read_excel(input_path, index_col='日期')
data_all = data_all.dropna()



In [128]:
print(len(data_all.index))

29550


In [132]:
data_ts = pd.get_dummies(data_all[['tm_mon','tm_wday','id']])
data_ts = data_ts.reset_index()
data_all = data_all.reset_index()
data_all = pd.merge(data_all,data_ts,how='left',on="日期")
data_all =data_all.set_index('日期')


KeyError: "['tm_mon', 'tm_wday'] not in index"

In [135]:
print(data_all)
print(data_ts)

KeyError: 'tm_mom3'

In [131]:

data_test = data_all[(data_all['tm_mon']==3)|
                     (data_all['tm_mon']==6)|
                    (data_all['tm_mon']==9)|
                    (data_all['tm_mon']==12)]
data_train = data_all[(data_all['tm_mon']==1)|
                     (data_all['tm_mon']==2)|
                    (data_all['tm_mon']==4)|
                    (data_all['tm_mon']==5)|
                     (data_all['tm_mon']==7)|
                    (data_all['tm_mon']==8)|
                    (data_all['tm_mon']==10)|
                     (data_all['tm_mon']==11)]

KeyError: 'tm_mon'

In [125]:
data_aod_test = data_test[['AOD_0',
                     'AOD_1',
                     'AOD_2',
                     'AOD_3',
                     'AOD_4',
                     'AOD_5',
                     'AOD_6',
                     'AOD_7',
                     'AOD_8',
                     'AOD_9',
                     'AOD_10',
                     'AOD_11',
                     'AOD_12',
                     'AOD_13',
                     'AOD_14',
                     'AOD_15',
                     'AOD_16']]
data_sky_test = data_test[['apparentTemperatureHigh',
                     'apparentTemperatureLow',
                     'apparentTemperatureMax',
                     'apparentTemperatureMin',
                     'cloudCover',
                     'dewPoint',
                     'humidity',
                     'moonPhase',
                     'ozone',
                     'precipAccumulation',
                     'precipIntensity',
                     'precipIntensityMax',
                     'pressure',
                     'sunriseTime',
                     'sunsetTime',
                     'temperatureHigh',
                     'temperatureLow',
                     'temperatureMax',
                     'temperatureMin',
                     'uvIndex',
                     'visibility',
                     'windBearing',
                     'windGust',
                     'windSpeed',
                     'apparentTemperature',
                     'temperature']]


data_ts = data_ts.set_index('日期')
data_ts_test = data_test[data_ts.columns]
# data_ts_test['id'] = data_ts_test['id'].map(lambda x: x.replace("A", ""))


data_t1_test = data_test[['AOD_0_T1',
                    'AOD_1_T1',
                    'apparentTemperatureHigh_T1',
                    'apparentTemperatureLow_T1',
                    'apparentTemperatureMax_T1',
                    'apparentTemperatureMin_T1',
                    'cloudCover_T1',
                    'dewPoint_T1',
                    'humidity_T1',
                    'moonPhase_T1',
                    'ozone_T1',
                    'precipAccumulation_T1',
                    'precipIntensity_T1',
                    'precipIntensityMax_T1',
                    'pressure_T1',
                    'sunriseTime_T1',
                    'sunsetTime_T1',
                    'temperatureHigh_T1',
                    'temperatureLow_T1',
                    'temperatureMax_T1',
                    'temperatureMin_T1',
                    'uvIndex_T1',
                    'visibility_T1',
                    'windBearing_T1',
                    'windGust_T1',
                    'windSpeed_T1',
                    'apparentTemperature_T1',
                    'temperature_T1']]
data_ndvi_test = data_test[['NDVI_0',
                      'NDVI_1',
                      'NDVI_2',
                      'NDVI_3',
                      'NDVI_4',
                      'NDVI_5',
                      'NDVI_6',
                      'NDVI_7',
                      'NDVI_8',
                      'NDVI_9',
                      'NDVI_10',
                      'NDVI_11',
                      'NDVI_12',
                      'NDVI_13',
                      'NDVI_14',
                      'NDVI_15',
                      'NDVI_16']]
data_ndvi_test = data_test[['NDVI_0']]
data_pm_test = data_test[['PM25']]

KeyError: '日期'

In [97]:
data_aod_train = data_train[['AOD_0',
                     'AOD_1',
                     'AOD_2',
                     'AOD_3',
                     'AOD_4',
                     'AOD_5',
                     'AOD_6',
                     'AOD_7',
                     'AOD_8',
                     'AOD_9',
                     'AOD_10',
                     'AOD_11',
                     'AOD_12',
                     'AOD_13',
                     'AOD_14',
                     'AOD_15',
                     'AOD_16']]
data_sky_train = data_train[['apparentTemperatureHigh',
                     'apparentTemperatureLow',
                     'apparentTemperatureMax',
                     'apparentTemperatureMin',
                     'cloudCover',
                     'dewPoint',
                     'humidity',
                     'moonPhase',
                     'ozone',
                     'precipAccumulation',
                     'precipIntensity',
                     'precipIntensityMax',
                     'pressure',
                     'sunriseTime',
                     'sunsetTime',
                     'temperatureHigh',
                     'temperatureLow',
                     'temperatureMax',
                     'temperatureMin',
                     'uvIndex',
                     'visibility',
                     'windBearing',
                     'windGust',
                     'windSpeed',
                     'apparentTemperature',
                     'temperature']]
data_ts_train = data_train[['tm_mon','tm_wday','id']]
data_ts_train['id'] = data_ts_train['id'].map(lambda x: x.replace("A", ""))
data_t1_train = data_train[['AOD_0_T1',
                    'AOD_1_T1',
                    'apparentTemperatureHigh_T1',
                    'apparentTemperatureLow_T1',
                    'apparentTemperatureMax_T1',
                    'apparentTemperatureMin_T1',
                    'cloudCover_T1',
                    'dewPoint_T1',
                    'humidity_T1',
                    'moonPhase_T1',
                    'ozone_T1',
                    'precipAccumulation_T1',
                    'precipIntensity_T1',
                    'precipIntensityMax_T1',
                    'pressure_T1',
                    'sunriseTime_T1',
                    'sunsetTime_T1',
                    'temperatureHigh_T1',
                    'temperatureLow_T1',
                    'temperatureMax_T1',
                    'temperatureMin_T1',
                    'uvIndex_T1',
                    'visibility_T1',
                    'windBearing_T1',
                    'windGust_T1',
                    'windSpeed_T1',
                    'apparentTemperature_T1',
                    'temperature_T1']]
data_ndvi_train = data_train[['NDVI_0']]
""",
                      'NDVI_1',
                      'NDVI_2',
                      'NDVI_3',
                      'NDVI_4',
                      'NDVI_5',
                      'NDVI_6',
                      'NDVI_7',
                      'NDVI_8',
                      'NDVI_9',
                      'NDVI_10',
                      'NDVI_11',
                      'NDVI_12',
                      'NDVI_13',
                      'NDVI_14',
                      'NDVI_15',
                      'NDVI_16'"""
data_pm_train = data_train[['PM25']]

D:\DataScience\Anaconda\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [98]:
for cname in data_ts_test.columns:
    data_ts_test[cname] = data_ts_test[cname].map(lambda x : str(x))

for cname in data_ts_train.columns:
    data_ts_train[cname] = data_ts_train[cname].map(lambda x : str(x))  

D:\DataScience\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\DataScience\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [101]:
"""
from sklearn.preprocessing import  OneHotEncoder
import numpy as np
data_ts_train = np.array(data_ts_train)
data_ts_test = np.array(data_ts_test)

data_ts_test = OneHotEncoder.fit_transform(data_ts_test)
data_ts_train = OneHotEncoder.fit_transform(data_ts_train)
"""
data_ts_train = pd.get_dummies(data_ts_train)
data_ts_test = pd.get_dummies(data_ts_test)

In [112]:
print(data_ts_test.shape)

print(data_ts_test.shape)

(9277, 126)
(9277, 126)


In [104]:
# 第一个模块 当日天气模块
#
# 输入层
Meteorology_input = Input(shape=(len(data_sky_test.columns),), name="Meteorology_input")
AODs_input = Input(shape=(len(data_aod_test.columns),), name="AODs_input")
# 因素融合层
meteorology_concat = concatenate([Meteorology_input, AODs_input])
# 全连接层
meteorology_x = Dense(
    24,
    activation='relu',
    name="FullConnectionMA_1")(meteorology_concat)
# 残差层
meteorology_residual_connection = Dense(
    48,
    activation='relu',
    name="ResidualConnectionMA")(meteorology_x)
meteorology_residual_connection = Dense(
    24,
    activation='relu',
    name="FullConnectionMA_RC")(meteorology_residual_connection)
meteorology_residual_connection = add(
    [meteorology_x, meteorology_residual_connection], name="ResidualConnectionMA_Add")
# 全连接层
meteorology_x = Dense(
    3,
    activation='relu',
    kernel_regularizer=keras.regularizers.l2(0.5),
    name="FullConnectionMA_2")(meteorology_residual_connection)
# Dropout
meteorology_y = core.Dropout(
    rate=0.25, name="Meteorology_Module")(meteorology_x)
# 模型层
model_meteorology = Model(
    inputs=[
        Meteorology_input,
        AODs_input],
    outputs=meteorology_y)

In [105]:
# 第二个模块 时滞天气模块

Weather_input = Input(shape=(len(data_t1_test.columns),), name="Weather_input")  # 只添加新的因素即可
# AODs_input = Input(shape=(17,), name="AODs_input")
# 因素融合层
weather_concat = concatenate([Weather_input, AODs_input])
# 全连接层
weather_x = Dense(
    24,
    activation='relu',
    name="FullConnectionWA_1")(weather_concat)
# 残差层
weather_residual_connection = Dense(
    36,
    activation='relu',
    name="ResidualConnectionWA")(weather_x)
weather_residual_connection = Dense(
    24,
    activation='relu',
    name="FullConnectionWAForRC")(weather_residual_connection)
weather_residual_connection = add(
    [weather_x, weather_residual_connection], name="ResidualConnectionWA_Add")
# 全连接层
weather_x = Dense(
    3,
    activation='relu',
    kernel_regularizer=keras.regularizers.l2(0.5),
    name="FullConnectionWA_2")(weather_residual_connection)
# Dropout
# use `rate` instead of `keep_prob`. r=1-p.
weather_y = core.Dropout(rate=0.25, name="Weather_Module")(weather_x)

# 模型层
model_weather = Model(inputs=[Weather_input, AODs_input], outputs=weather_y)



In [106]:
# 第三个模块 其他空气污染物模块 更改为ndvi
# 输入层
OtherPollution_input = Input(shape=(len(data_ndvi_test.columns),))
# 因素融合层
otherpollution_concat = concatenate([OtherPollution_input, AODs_input])
# 全连接层
otherpollution_x = Dense(
    4,
    activation='relu',
    name="FullConnectionOPA_1")(otherpollution_concat)
# 残差层
otherpollution_residual_connection = Dense(
    8,
    activation='relu',
    name="ResidualConnectionOPA")(otherpollution_x)
otherpollution_residual_connection = Dense(
    4,
    activation='relu',
    name="FullConnectionOPA_RC")(otherpollution_residual_connection)
otherpollution_residual_connection = add(
    [otherpollution_x, otherpollution_residual_connection], name="ResidualConnectionOPA_Add")
# 全连接层
otherpollution_x = Dense(
    3,
    activation='relu',
    kernel_regularizer=keras.regularizers.l2(0.5),
    name="FullConnectionOPA_2")(otherpollution_residual_connection)
# Dropout
otherpollution_y = core.Dropout(
    rate=0.5, name="OtherPollution_Module")(otherpollution_x)
# 模型层
model_otherpollution = Model(
    inputs=[
        OtherPollution_input,
        AODs_input],
    outputs=otherpollution_y)


In [113]:
###### 第四个模块 时空元属性模块
# 输入层
TimeStation_input = Input(shape=(len(data_ts_train),), name="TimeStation_input")
# 编码层, 该层之前需要自己对数据提前独特编码, 后记不用，可以直接嵌入字符串。
# 因素融合层
timestation_concat = concatenate([TimeStation_input, AODs_input])
# 全连接层
timestation_x = Dense(
    24,
    activation='relu',
    name="FullConnectionOTSA_1")(timestation_concat)
# 残差层
timestation_residual_connection = Dense(
    48,
    activation='relu',
    name="ResidualConnectionTSA")(timestation_x)
timestation_residual_connection = Dense(
    24,
    activation='relu',
    name="FullConnectionTSA_RC")(timestation_residual_connection)
timestation_residual_connection = add(
    [timestation_x, timestation_residual_connection], name="ResidualConnectionTSA_Add")
# 全连接层
timestation_x = Dense(
    24,
    activation='relu',
    name="FullConnectionTSA_2")(timestation_residual_connection)
timestation_x = Dense(
    3,
    activation='relu',
    kernel_regularizer=keras.regularizers.l2(0.15),
    name="FullConnectionTSA_222")(timestation_x)
# Dropout
timestation_y = core.Dropout(
    rate=0.5, name="TimeStation_Module")(timestation_x)
# 模型层
model_timestation = Model(
    inputs=[
        TimeStation_input,
        AODs_input],
    outputs=timestation_y)

In [114]:
# 第五个模块 整体影响模块
# 因素融合层
allin_concat = concatenate([Meteorology_input,
                            Weather_input,
                            OtherPollution_input,
                            TimeStation_input,
                            AODs_input])
# 全连接层
allin_x = Dense(
    24,
    activation='relu',
    name="FullConnectionAIA_1")(allin_concat)
# 残差层
allin_residual_connection = Dense(
    24,
    activation='relu',
    name="ResidualConnectionAIA")(allin_x)
allin_residual_connection = Dense(
    24,
    activation='relu',
    name="FullConnectionAIA_RC")(allin_residual_connection)
allin_residual_connection = add(
    [allin_x, allin_residual_connection], name="ResidualConnectionAIA_Add")
# 全连接层
allin_x = Dense(2, activation='relu',
                kernel_regularizer=keras.regularizers.l2(0.05),
                name="FullConnectionAIA_2")(allin_residual_connection)
# Dropout
allin_y = core.Dropout(rate=0.25, name="AllIn_Module")(allin_x)
# 模型层
model_allin = Model(
    inputs=[
        Meteorology_input,
        Weather_input,
        OtherPollution_input,
        TimeStation_input,
        AODs_input],
    outputs=allin_y)


In [115]:
# 最后的融合
# 捕捉的影响的融合层
res_concat = concatenate([model_meteorology.output,
                          model_weather.output,
                          model_otherpollution.output,
                          model_timestation.output,
                          model_allin.output])
# 全连接层
res_x = Dense(
    48,
    activation='relu',
    name="ResFullConnectionResModelForLast")(res_concat)
# 残差连接层
res_residual_connection = Dense(
    72,
    activation='relu',
    name="ResidualConnectionLast")(res_x)
res_residual_connection = Dense(
    48,
    activation='relu',
    name="FullConnectionLast_RC")(res_residual_connection)
res_residual_connection = add(
    [res_x, res_residual_connection], name="ResidualConnectionLast_Add")
# 全连接层
res_x = Dense(8, activation='relu',
              name="FullConnectionLast_2")(res_residual_connection)
# Dropout
res_y = core.Dropout(rate=0.25, name="Res_Module")(res_x)
# sigmoid 最终融合结果
res_outcome = Dense(
    1,
    activation='sigmoid',
    kernel_regularizer=keras.regularizers.l2(0.1),
    name="sigmoid_FC")(res_y)
# 编译最终模型
model_last = Model(
    inputs=[
        Meteorology_input,
        Weather_input,
        OtherPollution_input,
        TimeStation_input,
        AODs_input],
    outputs=res_outcome)
model_last.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), metrics=["accuracy"])


In [116]:

# print(model_last.summary())
model_last.fit([data_sky_train,data_t1_train,data_ndvi_train,data_ts_train,data_aod_train],data_pm_train,epochs = 200, 
               batch_size = 512)


ValueError: Error when checking input: expected TimeStation_input to have shape (20273,) but got array with shape (130,)

In [85]:
res = model_last.predict([data_sky_test,data_t1_test,data_ndvi_test,data_ts_test,data_aod_test])

ValueError: Error when checking input: expected TimeStation_input to have shape (11,) but got array with shape (126,)

In [ ]:
datares =res - data_pm_test
print(datares.sum())